In [1]:
#import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import json 
import csv
from datetime import datetime
from citipy import citipy as ctpy
from config import weather_api_key


In [2]:
#create set of random latitudes and longitudes
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)


In [3]:
#add the latitudes and longitudes to a list
coordinates=list(lat_lngs)

In [4]:
#create a list for holding the nearest cities
cities = []

#identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = ctpy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    #if the city is unique, add to the city list
    if city not in cities:
        cities.append(city)

In [6]:
#create empty list to hold weather data
city_data = []

#build basic URL for OpenWeatherMap
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

#loop through all cities in the list
for i, city in enumerate(cities):
    
    #create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    #run API request for each of the cities
    try:
        #parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        #parse out the needed data
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_humidity = city_weather['main']['humidity']
        city_clouds = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_country = city_weather['sys']['country']
        city_weather_desc = city_weather['weather'][0]['description']
        city_data.append({"City": city.title(),
                         "Lat": city_lat,
                         "Lng": city_lng,
                         "Max Temp": city_max_temp,
                         "Humidity": city_humidity,
                         "Cloudiness": city_clouds,
                         "Wind Speed": city_wind,
                         "Country": city_country,
                         "Current Description": city_weather_desc})
    except:
        pass
    


In [7]:
#change to dataframe
city_data_df = pd.DataFrame(city_data)


#change column order to desired format
new_column_order = ['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Current Description']
city_data_df = city_data_df[new_column_order]

city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Saint-Philippe,RE,-21.3585,55.7679,69.39,83,92,16.15,light rain
1,Kribi,CM,2.9500,9.9167,75.54,89,65,5.88,light rain
2,Bluff,NZ,-46.6000,168.3333,49.80,73,91,4.99,overcast clouds
3,Bredasdorp,ZA,-34.5322,20.0403,49.32,87,4,6.91,clear sky
4,Koutsouras,GR,35.0339,25.9469,82.53,52,0,14.67,clear sky
5,Lata,PT,40.1629,-8.3327,61.81,98,3,1.74,clear sky
6,Lorengau,PG,-2.0226,147.2712,84.61,71,22,4.07,few clouds
7,Mataura,NZ,-46.1927,168.8643,49.48,78,91,4.29,overcast clouds
8,Albany,US,42.6001,-73.9662,71.67,91,100,6.49,overcast clouds
9,Rikitea,PF,-23.1203,-134.9692,71.62,77,24,3.60,few clouds


In [8]:
#create output file 
output_file = "weather_data/cities.csv"

#export data to CSV file
city_data_df.to_csv(output_file, index_label="City_ID")